In [1]:
import json
from pprint import pprint
import pandas as pd
import re
import numpy as np
import ast
import operator
from textblob import TextBlob
from textblob.np_extractors import ConllExtractor
from textblob.taggers import NLTKTagger
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer("english")

mat=pd.read_csv('mat.txt', sep = '\t')

with open('content_comments_data.json') as data_file:    
    dataset = json.load(data_file)

In [2]:
#удаление 1 слова
def retract_words(data):
    for w in words1['Words']:
        for comm in data['Comms']:
            for i, word in enumerate(comm):
                if (comm[i]==w):
                    del comm[i]
                if (re.search(r'[^\w*\.\,\'\0-9\!\:\;\?\-\\\/\"]', word)): 
                    del comm[i]
                else:
                    continue
    return data

#удаление сочетания из 2 слов
def retract_two_words(data):
    for w in words2['Words']:
        for comm in data['Comms']:
            for i, word in enumerate(comm):
                    if i>0:
                        if ((comm[i].join(comm[i-1]))==w):
                            del comm[i]
                            del comm[i-1]
    return data

#удаление сочетания из 3 слов
def retract_three_words(data):
    for w in words3['Words']:
        for comm in data['Comms']:
            for i, word in enumerate(comm):
                    if i>1:
                        if (comm[i].join(comm[i-1])).join(comm[i-2])==w:
                            del comm[i]
                            del comm[i-1]
                            del comm[i-2]
    return data

In [3]:
def search_popular(dataset, Ind):   
    #запись комментариев
    comme, real, name = [], [], []
    for i in range(len(dataset[Ind]['comments'])):
        comme.append(dataset[Ind]['comments'][i]['text'])
    data=pd.DataFrame()
    data['Comments']=comme
    for comm in data.Comments:
        comm=TextBlob(comm)
        real.append((' '.join(comm.words.lower()).split()))
    data['Comms']=real
    
    #удаление "лишнего"
    data=retract_words(data)
    data=retract_two_words(data)
    data=retract_three_words(data)
    
    #словарь частот
    words_dict = dict()
    for i in range(data.shape[1]):
        for comm in data['Comms']:
                for i, word in enumerate(comm):
                    if word not in words_dict:
                        words_dict[word] = 0
                    else:
                        words_dict[word] += 1
    stems=[]
    words_dict1=words_dict.copy()

    for word in words_dict1:
        x=stemmer.stem(word.decode('utf-8'))
        if x in stems:
            del words_dict[word]
        else: 
            stems.append(x)
    
    #поиск самых частых слов (4 глубины)
    maxim, maxim1, maxim2, maxim3 = 0, 0, 0, 0
    for k in words_dict.values():
        if (k>maxim):
            maxim=k
    for k in words_dict.values():
        if (k>maxim1)&(k!=maxim):
            maxim1=k

    for k in words_dict.items():
        if ((k[1]==maxim)|(k[1]==maxim1))&(k[1]!=1):
            name.append(k[0])

    if len(name)<6:
        for k in words_dict.values():
            if (k>maxim2)&(k!=maxim)&(k!=maxim1):
                maxim2=k
        for k in words_dict.items():
            if (k[1]==maxim2)&(k[1]!=1):
                name.append(k[0])

    if len(name)<6:
        for k in words_dict.values():
            if (k>maxim3)&(k!=maxim)&(k!=maxim1)&(k!=maxim2):
                maxim3=k
        for k in words_dict.items():
            if (k[1]==maxim3)&(k[1]!=1):
                name.append(k[0])
    
    result, url = [], []
    result.append(name)
    memes=pd.DataFrame()
    url.append(dataset[Ind]['url'])
    memes['Url']=url
    memes['Frequent']=result
    
    return memes

In [8]:
frequent=pd.DataFrame(columns=('Url', 'Frequent'))
for i in range(len(dataset)):
    frequent=frequent.append(search_popular(dataset, i))
frequent.to_pickle('frequent.p')
frequent.head()

,Url,Frequent
0,http://img.ifcdn.com/images/b136b806781c3480c8...,"[right, guess, take, phasma]"
0,http://img.ifcdn.com/images/62ee7d15b1ff253e52...,"[elmo, back, looks, milk, black, rich, store, ..."
0,http://img.ifcdn.com/videos/337c7a1f2265c2ad1c...,"[tears, male, get, cry, women, feminists, men,..."
0,http://img.ifcdn.com/images/e39eea7eb4dc0f0320...,"[money, smart, everyone, eagles, need, play]"
0,http://img.ifcdn.com/images/1917b8fb8c20fd464f...,"[good, get, girl, first, top, girlfriend]"


In [ ]:
coms, comme, real_coms, real_comme, real, url, all_com = [], [], [], [], [], [], []
  
for Ind in range(len(dataset)):
    print Ind
    url.append(dataset[Ind]['url'])
    for i in range(len(dataset[Ind]['comments'])):
        real_comme.append(dataset[Ind]['comments'][i]['text'])
        com=TextBlob(dataset[Ind]['comments'][i]['text']).correct()
        comme.append(''.join(com.lower()))
        all_com.append(''.join(com.lower()))
    coms.append(comme)
    real_coms.append(real_comme)
    comme=[]
    real_comme=[]
data=pd.DataFrame(columns=('Url', 'Comments', 'Good Comments', 'Words', 'Normal Words'))
data['Url']=url
data['Comments']=real_coms
data['Good Comments']=coms

for Ind in range(len(data)):
    for comm in data.Comments[Ind]:
        comm=TextBlob(comm)
        if comm not in real: real.append((' '.join(comm.words.lower())))
    data['Words'][Ind]=real
    real=[]
data.head()

0
1
2
3
4
5


In [28]:
words, wrds = [], []
mats=list(mat['Words'])
for Ind in range(len(data)):
    for sent in data['Words'][Ind]:
        for i, comm in enumerate(re.split(r'\s+', sent)):
            if comm not in wrds:
                if comm not in mats: 
                    wrds.append(comm)
    data['Normal Words'][Ind]=wrds
    wrds=[]  
del data['Words']
data.to_pickle('normal_words.p')
data.head()

,Url,Comments,Normal Words
0,http://img.ifcdn.com/images/b136b806781c3480c8...,"[Oh and Phasma dies, Is the humor corny or pol...","[oh, and, phasma, dies, is, the, humor, corny,..."
1,http://img.ifcdn.com/images/62ee7d15b1ff253e52...,"[In prison, He just got back from the store, W...","[in, prison, he, just, got, back, from, the, s..."
2,http://img.ifcdn.com/videos/337c7a1f2265c2ad1c...,"[Fukin rekt, Baby batter, Featureworthy, More ...","[rekt, baby, batter, featureworthy, more, proo..."
3,http://img.ifcdn.com/images/e39eea7eb4dc0f0320...,"[How are you going to contact everyone?, Is th...","[how, are, you, going, to, contact, everyone, ..."
4,http://img.ifcdn.com/images/1917b8fb8c20fd464f...,"[Spoken like a true virgin, Yeah I did that an...","[spoken, like, a, true, virgin, yeah, i, did, ..."
